# RNN과 어텐션을 사용한 자연어 처리

**설정**

먼저 몇 개의 모듈을 임포트한다. 맷플롯립 그림을 저장하는 함수를 준비한다.

In [17]:
# 공통 모듈 임포트
import os
import matplotlib.pyplot as plt

# 그림을 저장할 위치
PROJECT_ROOT_DIR = '.'
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, 'images')
os.makedirs(IMAGES_PATH, exist_ok=True)


def save_fig(fig_id, tight_layout=True, fig_extension='png', resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + '.' + fig_extension)
    print(f'그림 저장 {fig_id}')
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, dpi=resolution, format=fig_extension)

## Char-RNN을 사용해 셰익스피어 문제 생성하기

**시퀀스를 셔플 윈도우 배치로 나누기**

예를 들어, 0~14까지 시퀀스를 2개씩 이동하면서 길이가 5인 윈도우로 나누어 본다(가령,`[0, 1, 2, 3, 4]`, `[2, 3, 4, 5, 6]`, 등). 그다음 이를 섞고 입력(처음 네 개의 스텝)과 타깃(마지막 네 개의 스텝)으로 나눈다(즉, `[2, 3, 4, 5, 6]`를 `[[2, 3, 4, 5], [3, 4, 5, 6]]`로 나눈다). 그다음 입력/타깃 쌍 세 개로 구성된 배치를 만든다:

In [66]:
import tensorflow as tf

n_steps = 5
dataset = tf.data.Dataset.from_tensor_slices(tf.range(15))
dataset = dataset.window(n_steps, 2, drop_remainder=True)
dataset = dataset.flat_map(lambda window: window.batch(n_steps))
dataset = dataset.shuffle(10).map(lambda window: (window[:-1], window[1:]))
dataset = dataset.batch(3).prefetch(tf.data.AUTOTUNE)
for index, (X_batch, Y_batch) in enumerate(dataset):
    print(f'{"_" * 20} Batch {index}\nX_batch\n{X_batch.numpy()}\n{"=" * 5}\nY_batch\n{Y_batch.numpy()}')

____________________ Batch 0
X_batch
[[ 2  3  4  5]
 [ 8  9 10 11]
 [ 6  7  8  9]]
=====
Y_batch
[[ 3  4  5  6]
 [ 9 10 11 12]
 [ 7  8  9 10]]
____________________ Batch 1
X_batch
[[ 0  1  2  3]
 [10 11 12 13]
 [ 4  5  6  7]]
=====
Y_batch
[[ 1  2  3  4]
 [11 12 13 14]
 [ 5  6  7  8]]


### 훈련 데이터셋 만들기

**데이터 로드하고 데이터셋 준비하기**

In [69]:
from tensorflow import keras

shakespeare_url = 'https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt'
filepath = keras.utils.get_file('shakespeare.txt', shakespeare_url)
with open(filepath) as f:
    shakespeare_text = f.read()

In [71]:
print(shakespeare_text[:148])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?



In [75]:
''.join(sorted(set(shakespeare_text.lower())))

"\n !$&',-.3:;?abcdefghijklmnopqrstuvwxyz"

In [76]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True)
tokenizer.fit_on_texts(shakespeare_text)

In [77]:
tokenizer.texts_to_sequences(['First'])

[[20, 6, 9, 8, 3]]

In [79]:
tokenizer.sequences_to_texts([[20, 6, 9, 8, 3]])

['f i r s t']

In [96]:
max_id = len(tokenizer.word_index)  # 고유한 문자 개수
dataset_size = tokenizer.document_count  # 전체 문자 개수

In [97]:
import numpy as np

[encoded] = np.array(tokenizer.texts_to_sequences([shakespeare_text])) - 1

### 순차 데이터셋을 나누는 방법

In [98]:
train_size = dataset_size * 90 // 100
dataset = tf.data.Dataset.from_tensor_slices(encoded[:train_size])